In [ ]:
# =============================================
# ✅ Imports and Setup
# =============================================
import optuna
from optuna import TrialPruned
from optuna.integration import XGBoostPruningCallback
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import logging
import warnings
warnings.filterwarnings("ignore")

# =============================================
# ✅ Logging Configuration
# =============================================
logging.basicConfig(
    filename="optuna_tuning.log",
    filemode="w",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
print("📜 Logging initialized — check 'optuna_tuning.log' for trial details.")

# =============================================
# ✅ Load Data
# =============================================
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

X = train.drop(columns=["WeightCategory", "id"])
y = train["WeightCategory"]
X_test = test.drop(columns=["id"])

# =============================================
# ✅ Encode categorical columns
# =============================================
cat_cols = X.select_dtypes(include="object").columns.tolist()
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    X_test[col] = le.transform(X_test[col])

# Encode target if not numeric
if y.dtype == "object":
    le_target = LabelEncoder()
    y = le_target.fit_transform(y)

# Train-validation split
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# =============================================
# ✅ Objective Function without Callbacks
# =============================================
def objective(trial):
    try:
        params = {
            "objective": "multi:softmax",
            "num_class": len(np.unique(y)),
            "eval_metric": "mlogloss",
            "n_estimators": trial.suggest_int("n_estimators", 100, 600),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "gamma": trial.suggest_float("gamma", 0.0, 0.5),
            "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 2.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.5, 2.0),
            "random_state": 42,
            "n_jobs": -1,
        }

        model = xgb.XGBClassifier(**params)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_valid, y_valid)],
            verbose=False
        )

        preds = model.predict(X_valid)
        acc = accuracy_score(y_valid, preds)
        logging.info(f"Trial {trial.number}: Accuracy={acc:.4f}, Params={params}")
        return acc

    except TrialPruned:
        logging.warning(f"Trial {trial.number} pruned due to poor performance.")
        raise

    except Exception as e:
        logging.error(f"Trial {trial.number} failed due to: {e}")
        return 0.0

# =============================================
# ✅ Study Configuration
# =============================================
study_name = "xgboost_tuning_study"
storage_name = f"sqlite:///{study_name}.db"

study = optuna.create_study(
    study_name=study_name,
    direction="maximize",
    storage=storage_name,
    load_if_exists=True,
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=25)
)

# =============================================
# ✅ Run Optimization
# =============================================
try:
    study.optimize(objective, n_trials=500, show_progress_bar=True, n_jobs=1)
except KeyboardInterrupt:
    print("🛑 Optimization manually stopped.")
except Exception as e:
    logging.error(f"Optimization interrupted: {e}")

# =============================================
# ✅ Save Best Model and Submission
# =============================================
best_params = study.best_params
logging.info(f"Best Trial: {study.best_trial.number}, Accuracy={study.best_value}, Params={best_params}")

print(f"\n✅ Best Accuracy: {study.best_value:.4f}")
print(f"🏆 Best Parameters: {best_params}")

# Train final model on full dataset
final_model = xgb.XGBClassifier(
    **best_params,
    objective="multi:softmax",
    num_class=len(np.unique(y))
)
final_model.fit(X, y)

predictions = final_model.predict(X_test)
# Decode target if LabelEncoder was used
if 'le_target' in globals():
    predictions = le_target.inverse_transform(predictions)

submission = pd.DataFrame({
    "id": test["id"],
    "WeightCategory": predictions
})
submission.to_csv("submission_optuna.csv", index=False)
print("📁 submission_optuna.csv saved successfully!")


📜 Logging initialized — check 'optuna_tuning.log' for trial details.


[I 2025-10-25 13:03:48,989] Using an existing study with name 'xgboost_tuning_study' instead of creating a new one.


  0%|          | 0/500 [00:00<?, ?it/s]

[I 2025-10-25 13:03:55,071] Trial 745 finished with value: 0.9034438364982298 and parameters: {'n_estimators': 311, 'learning_rate': 0.05827701246010952, 'max_depth': 4, 'subsample': 0.8123948665404667, 'colsample_bytree': 0.6227373900640558, 'gamma': 0.43475742584389165, 'reg_alpha': 1.6510698747281858, 'reg_lambda': 1.8595472103260697}. Best is trial 745 with value: 0.9034438364982298.
[I 2025-10-25 13:03:58,827] Trial 746 finished with value: 0.9015127132281944 and parameters: {'n_estimators': 313, 'learning_rate': 0.035997355606750395, 'max_depth': 4, 'subsample': 0.8085332799611605, 'colsample_bytree': 0.7773789856182173, 'gamma': 0.4358783570718417, 'reg_alpha': 1.6226607812170517, 'reg_lambda': 1.8727387843588632}. Best is trial 745 with value: 0.9034438364982298.
[I 2025-10-25 13:04:02,663] Trial 747 finished with value: 0.8986160283231412 and parameters: {'n_estimators': 308, 'learning_rate': 0.03559631823344987, 'max_depth': 4, 'subsample': 0.8117600123280232, 'colsample_bytr

# Task
Visualize the results of the Optuna study, evaluate the model, and analyze parameter relationships.

## Visualize optimization history

### Subtask:
Generate plots to visualize the optimization process, such as the objective value over trials and parameter importance.


**Reasoning**:
Import the necessary Optuna visualization functions and generate the optimization history and parameter importance plots.



In [ ]:
from optuna.visualization import plot_optimization_history, plot_param_importances

# Generate optimization history plot
history_plot = plot_optimization_history(study)
history_plot.show()

# Generate parameter importances plot
importance_plot = plot_param_importances(study)
importance_plot.show()

## Evaluate the model

### Subtask:
Evaluate the performance of the final model using appropriate metrics on the test set.


**Reasoning**:
Evaluate the performance of the final model on the test set by making predictions, decoding them, loading true labels, encoding them, and calculating accuracy.



## Analyze parameter relationships

### Subtask:
Explore how different hyperparameters affected the model's performance during the Optuna study.


**Reasoning**:
Generate and display the parallel coordinate plot and the slice plot to visualize the relationship between hyperparameters and the objective value.



In [ ]:
# Generate a parallel coordinate plot
parallel_plot = optuna.visualization.plot_parallel_coordinate(study)
parallel_plot.show()

# Generate a slice plot
slice_plot = optuna.visualization.plot_slice(study)
slice_plot.show()

## Summary:

### Data Analysis Key Findings

*   The Optuna study results were successfully visualized through optimization history and parameter importance plots.
*   The evaluation of the final model on the test set could not be performed due to the absence of true labels in the `test.csv` file.
*   Model performance was implicitly evaluated on a validation set during the Optuna hyperparameter tuning process.
*   Relationships between hyperparameters and model performance were visually explored using parallel coordinate and slice plots.

### Insights or Next Steps

*   To fully evaluate the model's performance on unseen data, obtain a test set with true labels.
*   Analyze the generated parameter relationship plots to gain deeper insights into the hyperparameter tuning process and identify potential areas for further optimization.
